In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import openpyxl
import re
import os
from variableUtils import *
import variableUtils
from Utils import *
from ClassUtils import *
from pprint import pprint
import json
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from reportlab.lib.pagesizes import letter, landscape, A4, A3
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, PageBreak, Paragraph, Spacer, Image
from reportlab.lib import colors
from matplotlib.backends.backend_pdf import PdfPages
from reportlab.platypus import Paragraph, Spacer, KeepTogether, KeepInFrame
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
import io
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from openpyxl.formatting.rule import FormulaRule
import PIL
import json

warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
print(sns.__version__)


In [ ]:
jsonfilepath = '2025\Michael\\assessment-data.json'
folder, file, ext = getFolderandFileName(jsonfilepath)
with open(jsonfilepath) as f:
    data = json.load(f)
# display(data)
print(json.dumps(data, indent=4))

In [ ]:
# Convert to a structured DataFrame per date
data_dict = {}
df_dict = {}

for entry in data["result"]:
    date = entry["date"]
    flattened_entry = {
        "student_name": entry["student_name"],
        "supervisor_name": entry["supervisor_name"],
        "cohort": entry["cohort"],
        "subject": entry["subject"],
        "type": entry["type"],
        **entry["student_data"],
        **entry["supervisor_data"]
    }
    
    if date not in data_dict:
        data_dict[date] = []
    
    data_dict[date].append(flattened_entry)

# Create and display separate DataFrames per date
for date, records in data_dict.items():
    df = pd.DataFrame(records)
    df_dict[date] = df
    # df.to_excel(f"{folder}\{date}.xlsx", index=False)


In [ ]:
# display one item of data_dict
key0 = list(data_dict.keys())[0]
print(json.dumps(data_dict[key0], indent=4))

### Standard Setting for Aditi

In [ ]:
##
# Reload necessary libraries after execution state reset
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Reload the Preparation and Restoration scores
file_path = "/mnt/data/Cleaned_Transposed_Workshopping_Checklist.xlsx"
xls = pd.ExcelFile(file_path)

# Load data
prep_df = xls.parse("Preparation")
rest_df = xls.parse("Restoration")

# Reload the importance ranking and difficulty tables
prep_data = {
    "Item": [
        "DEJ is caries-free *",
        "Principles of selective caries removal on pulpal surfaces satisfied *",
        "Sufficient and uniform layer of carious dentine remaining, limited to pulpal surfaces only",
        "Conservative in the mesio-distal dimension",
        "Conservative in the bucco-lingual dimension",
        "Depth not overextended nor less than 1.5mm",
        "Avoids excessive loss or weakening of tooth structure or compromising pulp space *",
        "Gingival contact cleared *",
        "Cavosurface margins free from irregularities and small areas of undermined enamel",
        "Free from large areas of unsupported enamel or large fractures*",
        "Internal surfaces smooth and rounded without minor roughness",
        "Internal surfaces smooth and rounded without obvious irregularities and/or gouges",
        "No or minimal scuffs/ scratches/ surface loss",
        "Avoids small damages to adjacent tooth and/or surrounding surfaces requiring tooth modification",
        "Avoids large damages to adjacent tooth and/or surrounding surfaces requiring restoration *"
    ],
    "Difficulty": [
        "HARD", "HARD", "HARD", "MODERATE", "MODERATE", "MODERATE", "MODERATE", "MODERATE",
        "HARD", "HARD", "HARD", "HARD", "MODERATE", "MODERATE", "MODERATE"
    ],
    "Importance Ranking": [10, 10, 6, 7, 7, 7, 10, 10, 4, 10, 2, 5, 2, 7, 10]
}

rest_data = {
    "Item": [
        "Smooth and well-polished",
        "Sub-surface porosities or voids absent",
        "Large porosities or voids absent*",
        "Smooth junction",
        "Free of minor excess/deficiency only detectable by dental explorer",
        "Free of major excess that warrants additional finishing and polishing",
        "No open margins *",
        "No overhangs that warrant significant finishing and polishing*",
        "Tooth anatomy recreated",
        "Contour not bulky (including marginal ridge)",
        "Contour not deficient (including marginal ridge)",
        "Establishes anatomical contact area *",
        "No or minimal scuffs/ scratches/ surface loss",
        "Avoids surface damage 2mm beyond restoration",
        "Avoids damage to adjacent tooth and/or surrounding surfaces requiring restorative correction *"
    ],
    "Difficulty": [
        "MODERATE", "MODERATE", "MODERATE", "MODERATE", "MODERATE", "MODERATE", "MODERATE",
        "MODERATE", "MODERATE", "MODERATE", "MODERATE", "MODERATE", "HARD", "HARD", "HARD"
    ],
    "Importance Ranking": [2, 7, 10, 8, 8, 8, 10, 10, 2, 5, 7, 10, 2, 5, 10]
}

prep_criteria_df = pd.DataFrame(prep_data)
rest_criteria_df = pd.DataFrame(rest_data)

# Add total score columns
prep_df["Total Score"] = prep_df.iloc[:, 2:].sum(axis=1)
rest_df["Total Score"] = rest_df.iloc[:, 2:].sum(axis=1)

# Maximum possible scores
max_prep_score = prep_df.iloc[:, 2:-1].shape[1]
max_rest_score = rest_df.iloc[:, 2:-1].shape[1]

# **Method 1: Fixed Percentage Threshold**
pass_threshold = 0.7  # 70% cutoff

prep_df["Fixed Pass"] = prep_df["Total Score"] >= (max_prep_score * pass_threshold)
rest_df["Fixed Pass"] = rest_df["Total Score"] >= (max_rest_score * pass_threshold)

# **Method 2: Weighted Scoring Based on Importance Ranking**
weighted_prep_scores = prep_df.iloc[:, 2:-2] * prep_criteria_df["Importance Ranking"].values
weighted_rest_scores = rest_df.iloc[:, 2:-2] * rest_criteria_df["Importance Ranking"].values

prep_df["Weighted Score"] = weighted_prep_scores.sum(axis=1)
rest_df["Weighted Score"] = weighted_rest_scores.sum(axis=1)

max_weighted_prep_score = prep_criteria_df["Importance Ranking"].sum()
max_weighted_rest_score = rest_criteria_df["Importance Ranking"].sum()

prep_df["Weighted Pass"] = prep_df["Weighted Score"] >= (max_weighted_prep_score * pass_threshold)
rest_df["Weighted Pass"] = rest_df["Weighted Score"] >= (max_weighted_rest_score * pass_threshold)

# **Method 3: Difficulty-Adjusted Scoring**
hard_criteria_prep = prep_criteria_df[prep_criteria_df["Difficulty"] == "HARD"].index + 2
hard_criteria_rest = rest_criteria_df[rest_criteria_df["Difficulty"] == "HARD"].index + 2

prep_df["Hard Criteria Score"] = prep_df.iloc[:, hard_criteria_prep].sum(axis=1)
rest_df["Hard Criteria Score"] = rest_df.iloc[:, hard_criteria_rest].sum(axis=1)

hard_pass_threshold = 0.6  # 60% of HARD criteria must be correct

prep_df["Hard Criteria Pass"] = prep_df["Hard Criteria Score"] >= (len(hard_criteria_prep) * hard_pass_threshold)
rest_df["Hard Criteria Pass"] = rest_df["Hard Criteria Score"] >= (len(hard_criteria_rest) * hard_pass_threshold)

# Save results for visualization
combined_pass_fail_df = prep_df[["Student", "Examiner", "Total Score", "Fixed Pass", "Weighted Score", "Weighted Pass", "Hard Criteria Score", "Hard Criteria Pass"]].merge(
    rest_df[["Student", "Examiner", "Total Score", "Fixed Pass", "Weighted Score", "Weighted Pass", "Hard Criteria Score", "Hard Criteria Pass"]],
    on=["Student", "Examiner"],
    suffixes=("_Preparation", "_Restoration")
)


# **Visualization of Pass/Fail Rates for Each Examiner**
plt.figure(figsize=(12, 6))
sns.countplot(x="Examiner", hue="Fixed Pass_Preparation", data=combined_pass_fail_df, palette="Blues")
plt.title("Pass Rate per Examiner (Fixed Threshold - Preparation)")
plt.xlabel("Examiner")
plt.ylabel("Number of Students")
plt.legend(title="Passed", labels=["Failed", "Passed"])
plt.show()

plt.figure(figsize=(12, 6))
sns.countplot(x="Examiner", hue="Fixed Pass_Restoration", data=combined_pass_fail_df, palette="Reds")
plt.title("Pass Rate per Examiner (Fixed Threshold - Restoration)")
plt.xlabel("Examiner")
plt.ylabel("Number of Students")
plt.legend(title="Passed", labels=["Failed", "Passed"])
plt.show()

plt.figure(figsize=(12, 6))
sns.countplot(x="Examiner", hue="Weighted Pass_Preparation", data=combined_pass_fail_df, palette="Blues")
plt.title("Pass Rate per Examiner (Weighted Scoring - Preparation)")
plt.xlabel("Examiner")
plt.ylabel("Number of Students")
plt.legend(title="Passed", labels=["Failed", "Passed"])
plt.show()

plt.figure(figsize=(12, 6))
sns.countplot(x="Examiner", hue="Weighted Pass_Restoration", data=combined_pass_fail_df, palette="Reds")
plt.title("Pass Rate per Examiner (Weighted Scoring - Restoration)")
plt.xlabel("Examiner")
plt.ylabel("Number of Students")
plt.legend(title="Passed", labels=["Failed", "Passed"])
plt.show()

plt.figure(figsize=(12, 6))
sns.countplot(x="Examiner", hue="Hard Criteria Pass_Preparation", data=combined_pass_fail_df, palette="Blues")
plt.title("Pass Rate per Examiner (Hard Criteria - Preparation)")
plt.xlabel("Examiner")
plt.ylabel("Number of Students")
plt.legend(title="Passed", labels=["Failed", "Passed"])
plt.show()

plt.figure(figsize=(12, 6))
sns.countplot(x="Examiner", hue="Hard Criteria Pass_Restoration", data=combined_pass_fail_df, palette="Reds")
plt.title("Pass Rate per Examiner (Hard Criteria - Restoration)")
plt.xlabel("Examiner")
plt.ylabel("Number of Students")
plt.legend(title="Passed", labels=["Failed", "Passed"])
plt.show()
